# Overview:
In this notebook, we create the graph, which forms the backbone of our algorithm. We use networkx library for graph manipulations. We use the timetable data to create the graph. Each node in our graph corresponds to a stop and each edge between two nodes corresponds to a particular trip. There can exists multiple directed edges between two nodes, corresponding to different trip_ids. We also add walking edges between stops which are closer than 500m. \
Each directed edge (from station a to station b) in our graph has the following edge attributes:
1. trip_id : trip_id from the timetable data or 'walk' for walking trips
2. dep_time: Departure time from station a
3. arr_time: Arrival time at station b
4. walking_time: Only for walking edges, time taken to walk from station a to station b, at 50m/1min.

In [ ]:
import pandas as pd
from hdfs3 import HDFileSystem
hdfs = HDFileSystem(user='ebouille')
import os
username = os.environ['JUPYTERHUB_USER']
import networkx as nx
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import pickle
import heapq
import pickle
import networkx as nx
import math

In [ ]:
def read_parquet_from_hdfs(filename):
    files = hdfs.glob(f'/user/{username}/{filename}.parquet/*.parquet')
    df = pd.DataFrame()
    for file in files:
        with hdfs.open(file) as f:
            df = df.append(pd.read_parquet(f))
    return df

In [ ]:
# Read relevant data
stops = read_parquet_from_hdfs("stops")
stop_times = read_parquet_from_hdfs("stop_times")
calendar = read_parquet_from_hdfs("calendar") 
routes = read_parquet_from_hdfs("routes")
trips = read_parquet_from_hdfs("trips")

def route_desc_to_product(x):
    if x == 'Bus':
        return u'bus'
    elif x == 'Tram':
        return u'tram'
    elif x == 'Taxi':
        return u''
    else:
        return 'zug'
    
routes['product_id'] = routes['route_desc'].map(route_desc_to_product)
route_to_product = {row['route_id']: row['product_id'] for _, row in routes.iterrows()}
trip_id_to_route = {row['trip_id']: row['route_id'] for _, row in trips.iterrows()}
trip_to_product = {trip: route_to_product[trip_id_to_route[trip]]
                  for trip in trips['trip_id'].unique()}
product_id_map = {
    u'': 0,
    u'bus': 1,
    u'tram': 2,
    u'zug': 3
}

## Graph-Creation

In [ ]:
# Create a Multiple Edge Directed Graph -> G(V, E)
G = nx.MultiDiGraph()

For each trip_id data frame, add edges from the source to destination with the scheduled arrival and departure time along with the trip id.
Nodes/Vertices are also created in this as `networkx` will add a node if no such node is present.

In [ ]:
# Function to add all edges corresponding to one trip_id to the graph 
def add_to_graph(df):
    df_sorted = df.sort_values(['stop_sequence'])
    stop_ids = df_sorted['stop_id']
    weekly_cal = str(df_sorted.iloc[0]['monday']) + str(df_sorted.iloc[0]['tuesday']) + str(df_sorted.iloc[0]['wednesday']) + str(df_sorted.iloc[0]['thursday']) + str(df_sorted.iloc[0]['friday']) + str(df_sorted.iloc[0]['saturday']) + str(df_sorted.iloc[0]['sunday']) 
    attr = [{'dep':str(i[0])+':'+str(i[1]), \
             'arr':str(i[2])+':'+str(i[3]), \
             'trip_id':df_sorted.iloc[0]['trip_id'], \
             'weekly': weekly_cal} \
            for i in zip(df_sorted['departure_hour'][:-1],\
                         df_sorted['departure_minute'][:-1],\
                         df_sorted['arrival_hour'][1:],\
                         df_sorted['arrival_minute'][1:]\
                        )]
    G.add_edges_from([*zip(stop_ids[0:-1],stop_ids[1:],attr)])

In [ ]:
# NOTE: Time intensive task, usually requires 4-5 minutes
stop_times\
    .merge(trips, on='trip_id', how='left')\
    .drop(columns=['drop_off_type', 'pickup_type', 'direction_id', 'route_id'])\
    .merge(calendar, on='service_id', how='left')\
    .drop(columns=['start_date', 'end_date'])\
    .groupby('trip_id')\
    .apply(add_to_graph)

Number of Nodes (Stations) in the Graph

In [ ]:
print(len(G.nodes()))

In [ ]:
print(len(G.edges))

#### Walking Edges

In [ ]:
def distance_between_coordinates_km(lat1, lon1, lat2, lon2):
    # Returns distance in km between two co-ordinates
    from math import sin, cos, sqrt, atan2, radians

    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(float(lat1))
    lon1 = radians(float(lon1))
    lat2 = radians(float(lat2))
    lon2 = radians(float(lon2))

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
for i in range(len(stops)-1):
    stop_i = stops.iloc[i]
    stops_rest = stops.iloc[i+1:]
    stops_rest['dist_i'] = stops_rest.apply(lambda x: distance_between_coordinates_km(stop_i['stop_lat'], stop_i['stop_lon'], x['stop_lat'], x['stop_lon']) , axis=1)
    stops_rest = stops_rest[stops_rest['dist_i']<=0.5]
    attr = [{'walking_time':i_ , 'trip_id':'walk'} for i_ in (stops_rest['dist_i']*1000.0)/50.0]
    G.add_edges_from([*zip([stop_i['stop_id']]*len(stops_rest),stops_rest['stop_id'],attr)])
    G.add_edges_from([*zip(stops_rest['stop_id'], [stop_i['stop_id']]*len(stops_rest),attr)])

In [ ]:
print(len(G.edges))

#### Saving the graph object into a pickle file

In [ ]:
dir = "/work/final-assignment-group-y/data/"
filename = "graph.pickle"
filename_reverse = "graph_reverse.pickle"

In [ ]:
with open(dir + filename, 'wb') as config_dictionary_file:
    pickle.dump(G, config_dictionary_file)
    
with open(dir + filename_reverse, 'wb') as config_dictionary_file:
    pickle.dump(G.reverse(copy=True), config_dictionary_file)